# EEG Biometrics


In [68]:
# Run this cell to load required libraries and mount your Drive folder
import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
import os
from sklearn.svm import SVC
import pandas as pd
import itertools

# drive.mount('/content/drive')

In [72]:

class pairwiseSVM:
  """
  Define the SVM class which will handle the pairwise manipulation, training & prediction
  """
  def __init__(self, C=1.0, kernel='rbf', degree=3, random_state=None):
    self.svm = SVC(C=C, kernel=kernel, degree=degree, random_state=random_state)

  def read_train_data(filename, label_col = 'labels', id_col = 'id'):
    """
    The training data may be read from file or supplied as a DF when fitting the classifier. 
    In both cases, it is assumed that is in DF format with column names, and includes class labels and trial IDs.
    """
    data = pd.read_csv(filename)

    # Separate the trial IDs, class labels and feature data into separate numpy arrays.
    train_id = np.array(data[id_col]) # Retrieve the trial IDs for each row & convert to numpy array.
    y_train = np.array(data[label_col]) # Retrieve the class labels for each row & convert to numpy array.
    # Retrieve the training features only and convert to numpy array.
    x_train = data.drop([label_col, id_col], axis=1)
    x_train = np.array(x_train)

    self.train_id = train_id
    self.y_train = y_train
    self.x_train = x_train

  def construct_pairs(self, x_train=None, y_train=None, x_test = None, y_test = None):
    """
    Method for constructing pairs from the training or testing data.
    """
    if (x_train is None) ^ (y_train is None):
      raise Exception("Both x_train and y_train datasets should be supplied, or neither.")
    elif x_train is None and y_train is None:
      x_train = self.x_train
      y_train = self.y_train


    # If x_test is not supplied, we want to construct all pairs of the training data with itself.
    if x_test is None:
      # Using the permutations function allows us to get symmetric pairs but excludes pairs of the same index. i.e. both (i,j) and (j,i) will be included but only where i!=j
      index_pairs = itertools.permutations(range(len(x_train)), 2) # Get all two-way permutations of the indexes.

      n_pairs = len(x_train)*len(x_train) - len(x_train) # All two-way combinations except where the indexes are the same.
      x_pairs = np.zeros((n_pairs, x_train.shape[1]*2)) # Create a blank array to hold the concatenated feature vector pairs.
      y_pairs = np.zeros(n_pairs) # Create a blank vector to hold class similarity flag.

      for count, (i,j) in enumerate(index_pairs):
        x_pairs[count] = np.concatenate((x_train[i],x_train[j])) # Concatenate the feature vectors for each pair.
        y_pairs[count] = y_train[i] == y_train[j] # Check if the pair comes from the same class or not.
    
    # If x_test is supplied, we want to construct all pairs combining the test data and the training data.
    elif x_test is not None and y_test is not None:
      index_pairs = itertools.product(range(len(x_train)), range(len(x_test))) # Get all two-way permutations of the indexes.

      n_pairs = len(x_train)*len(x_test) # Get the number of pairs.
      x_pairs = np.zeros((n_pairs, x_train.shape[1]*2)) # Create a blank array to hold the concatenated feature vector pairs.
      y_pairs = np.zeros(n_pairs) # Create a blank vector to hold class similarity flag.

      for count, (i,j) in enumerate(index_pairs):
        x_pairs[count] = np.concatenate((x_train[i],x_test[j])) # Concatenate the feature vectors for each pair.
        y_pairs[count] = y_train[i] == y_test[j] # Check if the pair comes from the same class or not.

    else: 
      raise Exception("If x_test is provided, then y_test must be provided too.")

    # Return the concatenated feature vectors for each pair, and the binary label whether they are from the same class.
    return x_pairs, y_pairs


  def fit(self, x_train = None, y_train = None):
    """
    Method to fit the SVM on the pairwise training data.
    """
    if (x_train is None) ^ (y_train is None):
      raise Exception("Either both the x_train and y_train datasets should be supplied, or neither.")

    # Get all pairwise combinations of the training data.
    elif x_train is None and y_train is None:
      x_pairs, y_pairs = self.construct_pairs()

    else:
      x_pairs, y_pairs = self.construct_pairs(x_train, y_train)

    print(x_pairs, y_pairs)


  def predict(self, x_test):
    """
    Predict class labels given a set of feature data.
    """
    pass
  
  def add_class(self, new_train, new_class):
    """
    Add new participant for prediction purposes.
    """
    pass

In [75]:
test = pairwiseSVM(C=1.0, kernel='rbf', degree=3, random_state=None) # Create a test instance of the class.

# Some small test data
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([[901,801,701],[602,603,604]])
y1 = np.array([1,2,3])
y2 = np.array([1,1])

# test.construct_pairs(a,y1,b,y2)
test.fit(a,y1)

[[1. 2. 3. 4. 5. 6.]
 [1. 2. 3. 7. 8. 9.]
 [4. 5. 6. 1. 2. 3.]
 [4. 5. 6. 7. 8. 9.]
 [7. 8. 9. 1. 2. 3.]
 [7. 8. 9. 4. 5. 6.]] [0. 0. 0. 0. 0. 0.]


In [ ]:
# Run this cell to save the changes

# drive.flush_and_unmount()
# print('All changes made in this colab session should now be visible in Drive.')